## Using Python to Perform Extract-Transform-Load (ETL Processing)
Modern Data Warehousing and Analytics solutions frequently use languages like Python or Scala to extract data from numerous sources, including relational database management systems, NoSQL database systems, real-time streaming endpoints and Data Lakes.  These languages can then be used to perform many types of transformation before then loading the data into a variety of destinations including file systems and data warehouses. This data can then be consumed by data scientists or business analysts.

In this lab you will recreate the **Northwind_DW** dimensional database from Lab 2; however, you'll take an entirely different approach. Instead of extracting, transforming and loading the date entirely on the database system entirely using SQL data definition language (DDL) and data manipulation language (DML) statements, here you will learn to interact with the RDBMS from a remote client running Python. You will learn to fetch data into Pandas DataFrames, perform all the necessary transformations in-memory on the client, and then push the newly transformed DataFrame back to the RDBMS using a Pandas function that will create the table and fill it with data with a single operation.

### Prerequisites:
#### Import the Necessary Libraries

In [1]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

#### Declare & Assign Connection Variables for the MySQL Server & Databases with which You'll be Working 

In [3]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = os.environ["PWD"]



src_dbname = "northwind"
dst_dbname = "northwind_dw"

#### Define Functions for Getting Data From and Setting Data Into Databases

In [102]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=360)
    connection = sqlEngine.connect()
        
    if db_operation == "insert":
        df.to_sql(table_name, schema = db_name, con = sqlEngine, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, schema=db_name,con=connection, index=False, if_exists='append')
    
    connection.close()

#### Create the New Data Warehouse database, and to Use it, Switch the Connection Context.
Clearly, you won't get very far without having a database to work with. Here we demonstrate how we can *drop* a database if it already exists, and then *create* the new **northwind_dw2** database and *use* it as the target of all subsequent operations.

In [5]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

with sqlEngine.connect() as conn:
    conn.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
    conn.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
    conn.execute(text(f"USE {dst_dbname};"))
    conn.close()

### 1.0. Create & Populate the Dimension Tables
#### 1.1. Extract Data from the Source Database Tables

In [6]:
sql_customers = "SELECT * FROM northwind.customers;"
df_customers = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_customers))
# df_customers.head(2)

In [7]:
sql_employees = "SELECT * FROM northwind.employees;"
df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_employees))
# df_employees.head(2)

In [8]:
sql_products = "SELECT * FROM northwind.products;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_products))
# df_products.head(2)

In [9]:
sql_shippers = "SELECT * FROM northwind.shippers;"
df_shippers = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_shippers))
# df_shippers.head(2)

#### 1.2. Perform Any Necessary Transformations
Pandas DataFrames enable extensive data modification capabilities. Here we will start by simply dropping features (columns) that we don't believe provide any real value to our analytics solution. Examples include columns having a high percentage of NULL values, columns having large amounts of free-text, and columns having binary large object (BLOB) data such as images or other documents. Then, we will rename the primary key column (id) to conform with data warehouse design standards.

In [10]:
drop_cols = ['email_address','home_phone','mobile_phone','web_page','notes','attachments']
df_customers.drop(drop_cols, axis=1, inplace=True)
df_customers.rename(columns={"id":"customer_key"}, inplace=True)

# df_customers.head(2)

In [11]:
drop_cols = ['mobile_phone','notes','attachments']
df_employees.drop(drop_cols, axis=1, inplace=True)
df_employees.rename(columns={"id":"employee_key"}, inplace=True)

# df_employees.head(2)

In [12]:
drop_cols = ['supplier_ids','description','attachments']
df_products.drop(drop_cols, axis=1, inplace=True)
df_products.rename(columns={"id":"product_key"}, inplace=True)

# df_products.head(2)

In [13]:
drop_cols = ['last_name','first_name','email_address','job_title','business_phone',
             'home_phone','mobile_phone','fax_number','web_page','notes','attachments']
df_shippers.drop(drop_cols, axis=1, inplace=True)
df_shippers.rename(columns={"id":"shipper_key"}, inplace=True)

# df_shippers.head(2)

#### 1.4. Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables
Here I demonstrate how we can create an iterable data structure containing the values needed to correctly create and populate the new dimension tables. If you inspect this code listing carefully, you'll notice that it's a **list** containing a **set** *(or vector)* for each dimension table. Each **set** then contains the *table_name* we need to assign to the table, the *pandas DataFrame* we crafted to define & populate the table, and the name we need to assign to the *primary_key* column.  With this *list of sets* defined, we can then call our **set_dataframe( )** function from within a **for *loop*** to create each *dimension* table.

In [14]:
db_operation = "insert"

tables = [('dim_customers', df_customers, 'customer_key'),
          ('dim_employees', df_employees, 'employee_key'),
          ('dim_products', df_products, 'product_key'),
          ('dim_shippers', df_shippers, 'shipper_key')]

In [15]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

### 2.0. Create & Populate the Fact Table
Here we will learn two approaches to creating the *fact_orders* fact table. The first approach demonstrates that a carefully crafted SQL SELECT statement can be used to perform this task... *but what fun would that be.* Seriously though, this approach is quick and effect if you already have the query, but what if you didn't have the opportunity to view and work with the data beforehand?  What's more, you may be required to combine data from multiple sources, some of which may not be relational database management systems. Then, a simple SQL query won't do!  You would need to load the data from the various sources (e.g., database tables, CSV or JSON files, NoSQL document collections, API stream data) and then combine them into a single dataframe that you could then use to create a new database table. For this reason we'll see how we can retrieve the data, but we won't bother to use it for creating a new table... we already know how to do that using the **set_dataframe( )** function anyway.

#### 2.1. First, you could simply use the SQL SELECT statement you authored in Lab 2 

In [32]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

with sqlEngine.connect() as conn:
    conn.execute(text(f"USE {dst_dbname};"))
    conn.execute(text("DROP TABLE IF EXISTS `fact_orders`;"))
    conn.execute(text("CREATE TABLE `fact_orders`(`fact_order_key` int NOT NULL AUTO_INCREMENT, `order_key` int, `order_status_id` int ,`order_details_id` int , `product_id` int DEFAULT NULL, `inventory_id` int DEFAULT NULL, `purchase_order_id` int DEFAULT NULL, `shipper_id` int DEFAULT NULL, `order_status_name` varchar(50) , `order_details_status_name` varchar(50) , `quantity` decimal(18,4) DEFAULT '0.0000', `unit_price` decimal(19,4) DEFAULT '0.0000', `discount` double DEFAULT '0', `order_date` datetime DEFAULT NULL, `shipped_date` datetime DEFAULT NULL, `ship_name` varchar(50) DEFAULT NULL,`ship_address` longtext, `ship_city` varchar(50) DEFAULT NULL, `ship_state_province` varchar(50) DEFAULT NULL, `ship_zip_postal_code` varchar(50) DEFAULT NULL, `ship_country_region` varchar(50) DEFAULT NULL, `shipping_fee` decimal(19,4) DEFAULT '0.0000', `taxes` decimal(19,4) DEFAULT '0.0000', `payment_type` varchar(50) DEFAULT NULL, `paid_date` datetime DEFAULT NULL, `tax_rate` double DEFAULT '0', PRIMARY KEY (`fact_order_key`), KEY `order_key` (`order_key`), KEY `order_status_id` (`order_status_id`), KEY `order_details_id` (`order_details_id`), KEY `product_id` (`product_id`), KEY `inventory_id` (`inventory_id`), KEY `purchase_order_id` (`purchase_order_id`), KEY `shipper_id` (`shipper_id`)) ENGINE=InnoDB AUTO_INCREMENT=82 DEFAULT CHARSET=utf8mb4;"))
    conn.execute(text("INSERT INTO `northwind_dw`.`fact_orders` ( `order_key`,`order_status_id`,`order_details_id`, `product_id` , `inventory_id` , `purchase_order_id` , `shipper_id` , `order_status_name`, `order_details_status_name`, `quantity` , `unit_price` , `discount` , `order_date`, `shipped_date`, `ship_name`, `ship_address`, `ship_city`, `ship_state_province`, `ship_zip_postal_code`, `ship_country_region`, `shipping_fee` , `taxes`, `payment_type`, `paid_date`, `tax_rate` )  SELECT o.`id`,  os.`id`,  od.`status_id`, od.`product_id` ,  od.`inventory_id` ,  od.`purchase_order_id` ,  o.`shipper_id` ,  os.`status_name`,  ods.`status_name`, od.`quantity` ,  od.`unit_price` ,  od.`discount` ,  o.`order_date`,  o.`shipped_date`,  o.`ship_name`,  o.`ship_address`,  o.`ship_city`,  o.`ship_state_province`,  o.`ship_zip_postal_code`,  o.`ship_country_region`,  o.`shipping_fee` ,  o.`taxes`,  o.`payment_type`,  o.`paid_date`,  o.`tax_rate`   FROM northwind.orders AS o  LEFT JOIN northwind.orders_status AS os  ON o.status_id=os.id  LEFT JOIN northwind.order_details as od  ON o.id=od.order_id  LEFT JOIN northwind.order_details_status as ods  ON od.status_id=ods.id;"))
    conn.close()

#### 2.2. Instead, Implement the solution using Pandas DataFrames to Craft the table
This is where we get to the point of this lab.  We'll query the source **northwind** database to fill a *dataframe* for each of the source tables we need to create our *fact_orders* fact table; orders, orders_status, order_details and order_details_status. Then, we'll learn how to *join* those *dataframes* using the **merge( )** method of the Pandas DataFrame.  We'll make any additional changes that we expect to see reflected in the *fact* table in our new MySQL database, and then, we'll push the *dataframe* back to the MySQL server to create and populate the new *fact* table.

##### 2.2.1. Get all the data from each of the four tables involved

In [40]:
sql_query = "SELECT * FROM northwind.orders;"
df_orders = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_query))
df_orders.head()

,id,employee_id,customer_id,order_date,shipped_date,shipper_id,ship_name,ship_address,ship_city,ship_state_province,ship_zip_postal_code,ship_country_region,shipping_fee,taxes,payment_type,paid_date,notes,tax_rate,tax_status_id,status_id
0,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,NV,99999,USA,200.0,0.0,Check,2006-01-15,None,0.0,None,3
1,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,NY,99999,USA,5.0,0.0,Credit Card,2006-01-20,None,0.0,None,3
2,32,4,12,2006-01-22,2006-01-22,2.0,John Edwards,123 12th Street,Las Vegas,NV,99999,USA,5.0,0.0,Credit Card,2006-01-22,None,0.0,None,3
3,33,6,8,2006-01-30,2006-01-31,3.0,Elizabeth Andersen,123 8th Street,Portland,OR,99999,USA,50.0,0.0,Credit Card,2006-01-30,None,0.0,None,3
4,34,9,4,2006-02-06,2006-02-07,3.0,Christina Lee,123 4th Street,New York,NY,99999,USA,4.0,0.0,Check,2006-02-06,None,0.0,None,3


In [41]:
sql_query = "SELECT * FROM northwind.orders_status;"
df_orders_status = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_query))
df_orders_status.head()

,id,status_name
0,0,New
1,1,Invoiced
2,2,Shipped
3,3,Closed


In [42]:
sql_query = "SELECT * FROM northwind.order_details;"
df_order_details = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_query))
df_order_details.head()

,id,order_id,product_id,quantity,unit_price,discount,status_id,date_allocated,purchase_order_id,inventory_id
0,27,30,34,100.0,14.0,0.0,2,None,96.0,83.0
1,28,30,80,30.0,3.5,0.0,2,None,NaN,63.0
2,29,31,7,10.0,30.0,0.0,2,None,NaN,64.0
3,30,31,51,10.0,53.0,0.0,2,None,NaN,65.0
4,31,31,80,10.0,3.5,0.0,2,None,NaN,66.0


In [43]:
sql_query = "SELECT * FROM northwind.order_details_status;"
df_order_details_status = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_query))
df_order_details_status.head()

,id,status_name
0,0,None
1,1,Allocated
2,2,Invoiced
3,3,Shipped
4,4,On Order


##### 2.2.2. Get the order_status column.
Here we use the dataframe's **merge( )** method to **inner join** the *orders* and the *orders_status* dataframes **on** the *status_id* column. We then use the dataframe's **rename( )** method to rename the *status_name* column to *order_status*, and use the dataframe's **drop( )** method to remove the *status_id* column.

In [105]:
df_o_os = pd.merge(df_orders, df_orders_status, left_on = "status_id", right_on = 'id', how = "inner", suffixes = ('', '_status'))
df_o_os.rename(columns = {"status_name": "order_status"}, inplace = True)
df_o_os.drop(columns = {'id_status', 'status_id'}, inplace = True)
df_o_os.head(2)

,id,employee_id,customer_id,order_date,shipped_date,shipper_id,ship_name,ship_address,ship_city,ship_state_province,ship_zip_postal_code,ship_country_region,shipping_fee,taxes,payment_type,paid_date,notes,tax_rate,tax_status_id,order_status
0,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,NV,99999,USA,200.0,0.0,Check,2006-01-15,None,0.0,None,Closed
1,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,NY,99999,USA,5.0,0.0,Credit Card,2006-01-20,None,0.0,None,Closed


##### 2.2.3. Get the order_details_status
Here we repeat the sequence of operations we used in the previous step to **inner join** the *order_details* and *order_details_status* dataframes for the sake of including the *order_details_status* column in place of the *status_id* column.

In [106]:
df_od_ods = pd.merge(df_order_details, df_order_details_status, left_on = "status_id", right_on = "id", how = "inner", suffixes = ('', '_status'))
df_od_ods.drop(columns = {"id_status"}, inplace=True)
df_od_ods.head(2)

,id,order_id,product_id,quantity,unit_price,discount,status_id,date_allocated,purchase_order_id,inventory_id,status_name
0,27,30,34,100.0,14.0,0.0,2,None,96.0,83.0,Invoiced
1,28,30,80,30.0,3.5,0.0,2,None,NaN,63.0,Invoiced


##### 2.2.4. Join the Orders and OrderDetails DataFrames
In this step we can now easily join the *orders* and *order_details* dataframes. Since each **order** (the *left* dataframe) can have many **order details** (the *right* dataframe), we'll need to implement a **right** *outer join* **on** the *order_id* column.

In [107]:
df_o_od = pd.merge(df_o_os, df_od_ods, left_on = "id", right_on = "order_id", how = "right", suffixes = ('_order', '_details'))
df_o_od.drop(columns = {"id_details"}, inplace=True)
df_o_od.head(2)

,id_order,employee_id,customer_id,order_date,shipped_date,shipper_id,ship_name,ship_address,ship_city,ship_state_province,...,order_id,product_id,quantity,unit_price,discount,status_id,date_allocated,purchase_order_id,inventory_id,status_name
0,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,NV,...,30,34,100.0,14.0,0.0,2,None,96.0,83.0,Invoiced
1,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,NV,...,30,80,30.0,3.5,0.0,2,None,NaN,63.0,Invoiced


##### 2.2.5. Perform any Additional Transformations
In this step we can prepare the DataFrame so that it defines exactly what we want to see created in the database.  Issues may include dropping unwanted columns, reordering the columns, and in our case, creating a new column to serve as the primary key.

In [108]:
df_o_od.insert(0, 'fact_order_key', range(1, len(df_o_od) + 1))
df_o_od.drop(columns = {"id_order"}, inplace = True)

In [110]:
order_id_col = df_o_od.pop("order_id")
df_o_od.insert(1, "order_id", order_id_col)

In [111]:
df_o_od.head(5)

,fact_order_key,order_id,employee_id,customer_id,order_date,shipped_date,shipper_id,ship_name,ship_address,ship_city,...,order_status,product_id,quantity,unit_price,discount,status_id,date_allocated,purchase_order_id,inventory_id,status_name
0,1,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,...,Closed,34,100.0,14.0,0.0,2,None,96.0,83.0,Invoiced
1,2,30,9,27,2006-01-15,2006-01-22,2.0,Karen Toh,789 27th Street,Las Vegas,...,Closed,80,30.0,3.5,0.0,2,None,NaN,63.0,Invoiced
2,3,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,...,Closed,7,10.0,30.0,0.0,2,None,NaN,64.0,Invoiced
3,4,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,...,Closed,51,10.0,53.0,0.0,2,None,NaN,65.0,Invoiced
4,5,31,3,4,2006-01-20,2006-01-22,1.0,Christina Lee,123 4th Street,New York,...,Closed,80,10.0,3.5,0.0,2,None,NaN,66.0,Invoiced


In [116]:
df_o_od.columns

Index(['fact_order_key', 'order_id', 'employee_id', 'customer_id',
       'order_date', 'shipped_date', 'shipper_id', 'ship_name', 'ship_address',
       'ship_city', 'ship_state_province', 'ship_zip_postal_code',
       'ship_country_region', 'shipping_fee', 'taxes', 'payment_type',
       'paid_date', 'notes', 'tax_rate', 'tax_status_id', 'order_status',
       'product_id', 'quantity', 'unit_price', 'discount', 'status_id',
       'date_allocated', 'purchase_order_id', 'inventory_id', 'status_name'],
      dtype='object')

##### 2.2.6. Write the DataFrame Back to the Database

In [114]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

conn = sqlEngine.connect()

conn.execute(text(f"USE {dst_dbname};"))
conn.execute(text("DROP TABLE IF EXISTS `fact_orders`;"))

In [115]:
tables = [('fact_orders', df_o_od, 'fact_order_key')]
db_operation = "insert"

for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

### 3.0. Demonstrate that the New Data Warehouse Exists and Contains the Correct Data
To demonstrate the viability of your solution, author a SQL SELECT statement that returns:
-	Each Customer’s Last Name
-	The total amount of the order quantity associated with each customer
-	The total amount of the order unit price associated with each customer

In [118]:
sql_query = "SELECT dim_customers.customer_key, fact_orders.customer_id, dim_customers.last_name, fact_orders.quantity, fact_orders.unit_price FROM northwind_dw.dim_customers LEFT JOIN northwind_dw.fact_orders ON dim_customers.customer_key = fact_orders.customer_id" 

In [119]:
df_test = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_query))

In [121]:
df_test.head(20)

,customer_key,customer_id,last_name,quantity,unit_price
0,1,1.0,Bedecs,25.0,2.99
1,1,1.0,Bedecs,25.0,46.00
2,1,1.0,Bedecs,25.0,18.00
3,1,1.0,Bedecs,40.0,18.40
4,2,NaN,Gratacos Solsona,NaN,NaN
5,3,3.0,Axen,0.0,38.00
6,3,3.0,Axen,0.0,2.99
7,3,3.0,Axen,3.0,40.00
8,3,3.0,Axen,50.0,10.00
9,3,3.0,Axen,200.0,9.65
